In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import tensorflow as tf
import random
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm

import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

import lightgbm
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

In [27]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/d/homaabedinzadeh/futuredays/90days.csv
/kaggle/input/d/homaabedinzadeh/holidayname/holidayname.csv
/kaggle/input/amirkabirdam/holiday.csv
/kaggle/input/amirkabirdam/dam12.csv
/kaggle/input/finaldb/finaldatset.csv
/kaggle/input/climate-indeices/ONI.xlsx
/kaggle/input/climate-indeices/ENSO.xlsx
/kaggle/input/climate-indeices/AMO.xlsx
/kaggle/input/climate-indeices/NAO.xlsx
/kaggle/input/climate-indeices/SOI.xlsx


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [73]:
df = pd.read_csv('/kaggle/input/finaldb/finaldatset.csv')
df.head(2)

,soi_shift_1,soi_shift_3,soi_shift_6,soi_shift_9,soi_shift_12,soi_rollingmean_3,soi_rollingstd_3,soi_rollingmin_3,soi_rollingmax_3,soi_div_rollingmean_3,soi_div_rollingstd_3,soi_div_rollingmin_3,soi_div_rollingmax_3,soi_rollingmean_4,soi_rollingstd_4,soi_rollingmin_4,soi_rollingmax_4,soi_div_rollingmean_4,soi_div_rollingstd_4,soi_div_rollingmin_4,soi_div_rollingmax_4,soi_rollingmean_5,soi_rollingstd_5,soi_rollingmin_5,soi_rollingmax_5,soi_div_rollingmean_5,soi_div_rollingstd_5,soi_div_rollingmin_5,soi_div_rollingmax_5,soi_rollingmean_6,soi_rollingstd_6,soi_rollingmin_6,soi_rollingmax_6,soi_div_rollingmean_6,soi_div_rollingstd_6,soi_div_rollingmin_6,soi_div_rollingmax_6,soi_rollingmean_7,soi_rollingstd_7,soi_rollingmin_7,soi_rollingmax_7,soi_div_rollingmean_7,soi_div_rollingstd_7,soi_div_rollingmin_7,soi_div_rollingmax_7,soi_rollingmean_8,soi_rollingstd_8,soi_rollingmin_8,soi_rollingmax_8,soi_div_rollingmean_8,soi_div_rollingstd_8,soi_div_rollingmin_8,soi_div_rollingmax_8,soi_rollingmean_9,soi_rollingstd_9,soi_rollingmin_9,soi_rollingmax_9,soi_div_rollingmean_9,soi_div_rollingstd_9,soi_div_rollingmin_9,soi_div_rollingmax_9,soi_rollingmean_10,soi_rollingstd_10,soi_rollingmin_10,soi_rollingmax_10,soi_div_rollingmean_10,soi_div_rollingstd_10,soi_div_rollingmin_10,soi_div_rollingmax_10,soi_rollingmean_11,soi_rollingstd_11,soi_rollingmin_11,soi_rollingmax_11,soi_div_rollingmean_11,soi_div_rollingstd_11,soi_div_rollingmin_11,soi_div_rollingmax_11,soi_rollingmean_12,soi_rollingstd_12,soi_rollingmin_12,soi_rollingmax_12,soi_div_rollingmean_12,soi_div_rollingstd_12,soi_div_rollingmin_12,soi_div_rollingmax_12,nao_shift_1,nao_shift_3,nao_shift_6,nao_shift_9,nao_shift_12,nao_rollingmean_3,nao_rollingstd_3,nao_rollingmin_3,nao_rollingmax_3,nao_div_rollingmean_3,nao_div_rollingstd_3,nao_div_rollingmin_3,nao_div_rollingmax_3,nao_rollingmean_4,nao_rollingstd_4,nao_rollingmin_4,nao_rollingmax_4,nao_div_rollingmean_4,nao_div_rollingstd_4,nao_div_rollingmin_4,nao_div_rollingmax_4,nao_rollingmean_5,nao_rollingstd_5,nao_rollingmin_5,nao_rollingmax_5,nao_div_rollingmean_5,nao_div_rollingstd_5,nao_div_rollingmin_5,nao_div_rollingmax_5,nao_rollingmean_6,nao_rollingstd_6,nao_rollingmin_6,nao_rollingmax_6,nao_div_rollingmean_6,nao_div_rollingstd_6,nao_div_rollingmin_6,nao_div_rollingmax_6,nao_rollingmean_7,nao_rollingstd_7,nao_rollingmin_7,nao_rollingmax_7,nao_div_rollingmean_7,nao_div_rollingstd_7,nao_div_rollingmin_7,nao_div_rollingmax_7,nao_rollingmean_8,nao_rollingstd_8,nao_rollingmin_8,nao_rollingmax_8,nao_div_rollingmean_8,nao_div_rollingstd_8,nao_div_rollingmin_8,nao_div_rollingmax_8,nao_rollingmean_9,nao_rollingstd_9,nao_rollingmin_9,nao_rollingmax_9,nao_div_rollingmean_9,nao_div_rollingstd_9,nao_div_rollingmin_9,nao_div_rollingmax_9,nao_rollingmean_10,nao_rollingstd_10,nao_rollingmin_10,nao_rollingmax_10,nao_div_rollingmean_10,nao_div_rollingstd_10,nao_div_rollingmin_10,nao_div_rollingmax_10,nao_rollingmean_11,nao_rollingstd_11,nao_rollingmin_11,nao_rollingmax_11,nao_div_rollingmean_11,nao_div_rollingstd_11,nao_div_rollingmin_11,nao_div_rollingmax_11,nao_rollingmean_12,nao_rollingstd_12,nao_rollingmin_12,nao_rollingmax_12,nao_div_rollingmean_12,nao_div_rollingstd_12,nao_div_rollingmin_12,nao_div_rollingmax_12,amo_shift_1,amo_shift_3,amo_shift_6,amo_shift_9,amo_shift_12,amo_rollingmean_3,amo_rollingstd_3,amo_rollingmin_3,amo_rollingmax_3,amo_div_rollingmean_3,amo_div_rollingstd_3,amo_div_rollingmin_3,amo_div_rollingmax_3,amo_rollingmean_4,amo_rollingstd_4,amo_rollingmin_4,amo_rollingmax_4,amo_div_rollingmean_4,amo_div_rollingstd_4,amo_div_rollingmin_4,amo_div_rollingmax_4,amo_rollingmean_5,amo_rollingstd_5,amo_rollingmin_5,amo_rollingmax_5,amo_div_rollingmean_5,amo_div_rollingstd_5,amo_div_rollingmin_5,amo_div_rollingmax_5,amo_rollingmean_6,amo_rollingstd_6,amo_rollingmin_6,amo_rollingmax_6,amo_div_rollingmean_6,amo_div_rollingstd_6,amo_div_rollingmin_6,amo_div_rollingmax_6,amo_rollingmean_7,amo_rollingstd_7,amo_rollingmin_7,amo_rollingmax_7,amo

In [83]:
horizon=-15

In [84]:
target = df['Total_in'].shift(horizon)

target.dropna(inplace=True)

In [85]:
y=target

x = df[:horizon]
#. drop(target,axis=1)

In [86]:
x.shape

(2922, 820)

In [87]:
len(y)

2922

In [88]:
x.replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_33/1102276997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.replace([np.inf, -np.inf], np.nan, inplace=True)


In [89]:
x.fillna(0, inplace=True)

/tmp/ipykernel_33/1828275282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.fillna(0, inplace=True)


In [104]:
tscv = TimeSeriesSplit(n_splits = 10)
test_rmse = []
train_rmse= []
test_r2 = []
train_r2=[]
i=0
for train_index, test_index in tscv.split(x):
    i+=1
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    if i>=6:
        
        model = xgb.XGBRegressor(n_estimators=460, learning_rate=0.01,  max_depth= 11 ,colsample_bytree= 0.6,  n_jobs=4)
        
        model.fit(X_train,Y_train)

    # Predictions
        test_predict = model.predict(X_test)
        train_predict = model.predict(X_train)
        
        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
        train_rmse.append ( np.sqrt(mean_squared_error(Y_train, train_predict)))
        
        test_r2 .append ( r2_score(Y_test, test_predict))
        train_r2 .append ( r2_score(Y_train, train_predict))
        
print("rmsetrain:",np.mean(train_rmse),"rmsetest",np.mean(test_rmse))
print ("r2train:",np.mean(train_r2),'r2test:',np.mean(test_r2) )





rmsetrain: 0.0441132666994906 rmsetest 0.5026929011040044
r2train: 0.9986936492345244 r2test: 0.452751405758149


440-90
r2train: 0.9979125411025691 r2test: 0.4354462088639449

# n_estimators=500, learning_rate=0.01,  max_depth= 11, colsample_bytree= 0.6
rmsetrain: 0.03573009459518882 rmsetest 0.5030607598981425
r2train: 0.9991428695186881 r2test: 0.4491554133701202



# n_estimators=480, learning_rate=0.01,  max_depth= 11 ,colsample_bytree= 0.6,  n_jobs=4

rmsetrain: 0.03963914254745377 rmsetest 0.5029176451344852
r2train: 0.9989450739031536 r2test: 0.45082075663583954


imp = pd.DataFrame({'Importance':model.feature_importances_,'Feature':x.columns})
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1)
sns.barplot(x="Importance", y="Feature", data=imp.sort_values(by="Importance", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [82]:

imp = pd.DataFrame({'Importance':model.feature_importances_,'Feature':x.columns})
fe=imp.sort_values(by='Importance',ascending=False).head(50)
fe.head(50)

,Importance,Feature
773,0.200289,Total_in_rollingmax_30
806,0.102053,Total_in_div_rollingmean_365
762,0.085414,Total_in_rollingmean_7
817,0.070968,Total_in
437,0.040523,TotalAbBarfVBaran_in_div_rollingmax_60
765,0.040030,Total_in_rollingmax_7
379,0.039371,Barandegi_in_div_rollingmax_60
752,0.021143,Total_in_shift_1
737,0.018516,humidity_rollingstd_180
736,0.014597,humidity_rollingmean_180


In [ ]:
tscv = TimeSeriesSplit(n_splits = 10)
test_rmse = []
train_rmse= []
test_r2 = []
train_r2=[]
i=0
for train_index, test_index in tscv.split(x):
    i+=1
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    if i>=6:
        lgb = lightgbm.LGBMRegressor( n_estimators=410, num_leaves=31, learning_rate=0.01, colsample_bytree=0.85,verbose=0)

        lgb.fit(X_train, Y_train)

        test_predict = lgb.predict(X_test, num_iteration=lgb.best_iteration_)
        train_predict = lgb.predict(X_train, num_iteration=lgb.best_iteration_)

        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
        train_rmse.append ( np.sqrt(mean_squared_error(Y_train, train_predict)))

        test_r2 .append ( r2_score(Y_test, test_predict))
        train_r2 .append ( r2_score(Y_train, train_predict))

print("rmsetrain:",np.mean(train_rmse),"rmsetest",np.mean(test_rmse))
print ("r2train:",np.mean(train_r2),'r2test:',np.mean(test_r2) )


In [ ]:

final.dropna(inplace=True, axis=0)
final.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(final)
scaled_df = pd.DataFrame(scaled_data, columns=final.columns)

final.reset_index(inplace=True ,drop=True)

In [ ]:

y=scaled_df[target]

x = scaled_df. drop(columns =[ target ],axis=1)#,'index'

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
tscv = TimeSeriesSplit(n_splits = 10)
test_rmse = []
train_rmse= []
test_r2 = []
train_r2=[]
i=0
for train_index, test_index in tscv.split(x):
    i+=1
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    if i>=6:
        
        mlp = MLPRegressor(hidden_layer_sizes=(83,15), activation='relu', solver='adam', max_iter=300,learning_rate='constant', early_stopping=True,alpha=  0.01)

        mlp.fit(X_train, Y_train)

        test_predict = mlp.predict(X_test)
        train_predict = mlp.predict(X_train)
        
        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
        train_rmse.append ( np.sqrt(mean_squared_error(Y_train, train_predict)))
        
        test_r2 .append ( r2_score(Y_test, test_predict))
        train_r2 .append ( r2_score(Y_train, train_predict))
        
print("rmsetrain:",np.mean(train_rmse),"rmsetest",np.mean(test_rmse))
print ("r2train:",np.mean(train_r2),'r2test:',np.mean(test_r2) )


## ی سوال: اینجا که ما شافل نداریم. درسته که توی ارزیابی مقادیر تغییر می کنه؟؟ در صورتی که شرایط کاملا یکسان و بدون تغییر هست.؟؟؟

In [36]:
target

0        0.315946
1        0.343056
2        0.305572
3        0.319793
4        0.308157
5        0.343265
6        0.315722
7        0.283333
8        0.294673
9        0.291387
10       0.285176
11       0.258125
12       0.290446
13       0.307608
14       0.280690
15       0.273307
16       0.296940
17       0.289606
18       0.256666
19       0.206846
20       0.228371
21       0.204451
22       0.202087
23       0.225958
24       0.197215
25       0.215391
26       0.213749
27       0.197519
28       0.252646
29       0.379067
30       0.299252
31       0.343200
32       0.344031
33       0.360840
34       0.330200
35       0.303820
36       0.273566
37       0.331179
38       0.331139
39       0.397412
40       0.819384
41       0.938558
42       2.234121
43       0.838747
44       0.670800
45       0.622160
46       0.584840
47       0.672220
48       0.695360
49       0.545596
50       0.615590
51       0.562400
52       0.480740
53       0.420260
54       0.457060
55       0

In [ ]:
scv = TimeSeriesSplit(n_splits = 10)
test_rmse = []
train_rmse= []
test_r2 = []
train_r2=[]
i=0
for train_index, test_index in tscv.split(x):
    i+=1
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    if i>=6:
        
        svr = SVR(kernel='rbf', C=1.0, epsilon=0.001)

        # آموزش مدل
        svr.fit(X_train, Y_train)

        # پیش‌بینی
        test_predict = svr.predict(X_test)
        train_predict = svr.predict(X_train)
        
        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
        train_rmse.append ( np.sqrt(mean_squared_error(Y_train, train_predict)))
        
        test_r2 .append ( r2_score(Y_test, test_predict))
        train_r2 .append ( r2_score(Y_train, train_predict))
        
print("rmsetrain:",np.mean(train_rmse),"rmsetest",np.mean(test_rmse))
print ("r2train:",np.mean(train_r2),'r2test:',np.mean(test_r2) )